In [1]:
!pip install nltk
!pip install stopwords
!pip install requests beautifulsoup4 nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!python -m nltk.downloader wordnet

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Generate** **Document**

In [23]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Function to generate text for a specific field
def generate_document(prompt, max_length=200, temperature=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, temperature=temperature)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Define fields and their prompts
fields = {
    "finance": "Navigating the complexities of modern finance.",
    "education": "Revolutionizing education through technology.",
    "environment": "Protecting the environment for a sustainable future.",
}

generated_documents = {}

# Generate documents for different fields
for field, prompt in fields.items():
    generated_documents[field] = generate_document(prompt)

# Post-processing to remove duplicate consecutive lines
for field in generated_documents:
    lines = generated_documents[field].split('\n')
    unique_lines = [lines[0]]
    for line in lines[1:]:
        if line != unique_lines[-1]:
            unique_lines.append(line)
    generated_documents[field] = '\n'.join(unique_lines)

# Print generated documents
for field, document in generated_documents.items():
    print(f"--- {field.capitalize()} ---")
    print(document)
    print("\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pa

--- Finance ---
Navigating the complexities of modern finance.

The first step is to understand the financial system. The financial system is a complex system that is complex because it is based on a set of rules and regulations. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules and regulations of the financial system. The rules and regulations are based on the rules a

**Processing** **on** **data**

In [27]:
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text
import pandas as pd

# Step 1: Processing the generated documents
processed_documents = {}
lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()

for field, document in generated_documents.items():
    # Cleaning
    cleaned_text = ''.join([char if char.isalnum() or char.isspace() else ' ' for char in document])

    # Normalization
    normalized_text = cleaned_text.lower()

    # Tokenization
    tokens = normalized_text.split()

    # Lemmatization and Stop words removal
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in text.ENGLISH_STOP_WORDS]

    # Stemming
    stemmed_tokens = [porter_stemmer.stem(token) for token in lemmatized_tokens]

    # Filter out words that have no meaning in WordNet
    meaningful_words = [word for word in stemmed_tokens if wordnet.synsets(word)]

    # Filter out short words
    meaningful_words = [word for word in meaningful_words if len(word) > 2]

    processed_documents[field] = ' '.join(meaningful_words)

    # Print unique words
    unique_words = set(meaningful_words)
    print("Unique words in document '{}': {}".format(field, unique_words))


Unique words in document 'finance': {'modern', 'base', 'set', 'step', 'rule', 'understand', 'complex'}
Unique words in document 'education': {'base', 'revolution', 'new', 'poor', 'step', 'american', 'process', 'child', 'rich'}
Unique words in document 'environment': {'environment', 'environ', 'nation', 'epa', 'protect', 'sustain', 'largest'}


# **TF  IDF   From scratch**

**Get** **TF** **for** **each** **word** **for** **all** **documents** tf=(freq of word in doc)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
# Step 2: Calculate TF for each word in all documents
corpus = list(processed_documents.values())
vectorizer = CountVectorizer()
term_counts = vectorizer.fit_transform(corpus)

# Get the feature names (unique words)
feature_names = vectorizer.get_feature_names_out()

# Print TF for each word in all documents
for i, word in enumerate(feature_names):
    tf_values = term_counts[:, i].toarray().flatten()
# Convert the matrix to DataFrame
tf_df = pd.DataFrame(term_counts.toarray(), columns=feature_names, index=processed_documents.keys())

# Print the TF table
print("Term Frequency (TF) for each word in all documents:")
print(tf_df)

Term Frequency (TF) for each word in all documents:
             american  base  child  complex  environ  environment  epa  \
finance             0    11      0        3        0            0    0   
education           2     1     12        0        0            0    0   
environment         0     0      0        0       15            1   13   

             largest  modern  nation  ...  poor  process  protect  revolution  \
finance            0       1       0  ...     0        0        0           0   
education          0       0       0  ...     9        1        0           1   
environment        1       0       1  ...     0        0       15           0   

             rich  rule  set  step  sustain  understand  
finance         0    21    1     1        0           1  
education       2     0    0     1        0           0  
environment     0     0    0     0       14           0  

[3 rows x 21 columns]


**Get** **IDF** **for** **`each`** **word** idf=log((1+n)/(1+nt))+1

In [29]:
# Calculate the document frequency (number of documents containing each word)
document_frequency = np.sum(term_counts > 0, axis=0)

# Calculate IDF for each word
total_documents = len(corpus )
idf = {}
for word, word_idx in zip(feature_names, range(len(feature_names))):
    word_df = document_frequency[0, word_idx]
    idf[word] = np.log((total_documents + 1) / (word_df + 1)) + 1   #n=total_documents ,nt=word_df

# Print IDF for each word
print("Inverse Document Frequency (IDF) for each word:")
for word, idf_value in idf.items():
    print("{}: {}".format(word, idf_value))


Inverse Document Frequency (IDF) for each word:
american: 1.6931471805599454
base: 1.2876820724517808
child: 1.6931471805599454
complex: 1.6931471805599454
environ: 1.6931471805599454
environment: 1.6931471805599454
epa: 1.6931471805599454
largest: 1.6931471805599454
modern: 1.6931471805599454
nation: 1.6931471805599454
new: 1.6931471805599454
poor: 1.6931471805599454
process: 1.6931471805599454
protect: 1.6931471805599454
revolution: 1.6931471805599454
rich: 1.6931471805599454
rule: 1.6931471805599454
set: 1.6931471805599454
step: 1.2876820724517808
sustain: 1.6931471805599454
understand: 1.6931471805599454


**Multiply**  **TF** * **IDF**

In [30]:
# Create a DataFrame to store TF * IDF values
tf_idf_df = pd.DataFrame(index=processed_documents.keys(), columns=feature_names)

# Multiply TF * IDF and store the result in the DataFrame
for word in feature_names:
    tf_values = tf_df[word].values  # TF values for the word
    idf_value = idf[word]  # IDF value for the word

    tf_idf_values = tf_values * idf_value  # Multiply TF with IDF
    tf_idf_df[word] = tf_idf_values

# Print the DataFrame
print("TF * IDF for each word in all documents:")
print(tf_idf_df)





TF * IDF for each word in all documents:
             american       base      child   complex    environ  environment  \
finance      0.000000  14.164503   0.000000  5.079442   0.000000     0.000000   
education    3.386294   1.287682  20.317766  0.000000   0.000000     0.000000   
environment  0.000000   0.000000   0.000000  0.000000  25.397208     1.693147   

                   epa   largest    modern    nation  ...       poor  \
finance       0.000000  0.000000  1.693147  0.000000  ...   0.000000   
education     0.000000  0.000000  0.000000  0.000000  ...  15.238325   
environment  22.010913  1.693147  0.000000  1.693147  ...   0.000000   

              process    protect  revolution      rich       rule       set  \
finance      0.000000   0.000000    0.000000  0.000000  35.556091  1.693147   
education    1.693147   0.000000    1.693147  3.386294   0.000000  0.000000   
environment  0.000000  25.397208    0.000000  0.000000   0.000000  0.000000   

                 step    sus

**Get** **Normalized** **TFIDF** =tf-idf(t,d,D)/sqrt(sum of squares of all tf-idf(t,d))

In [31]:
# Compute the sum of squares of all TF-IDF scores for each document
sum_of_squares = np.sqrt(np.sum(np.square(tf_idf_df), axis=1))

# Normalize TF-IDF scores for each document
normalized_tf_idf_df= tf_idf_df.div(sum_of_squares, axis=0)

# Print the normalized TF-IDF table
print("Normalized TF-IDF (Term Frequency-Inverse Document Frequency) for each word in all documents:")
print(normalized_tf_idf_df)





Normalized TF-IDF (Term Frequency-Inverse Document Frequency) for each word in all documents:
             american      base     child  complex   environ  environment  \
finance      0.000000  0.365613  0.000000  0.13111  0.000000     0.000000   
education    0.129871  0.049385  0.779226  0.00000  0.000000     0.000000   
environment  0.000000  0.000000  0.000000  0.00000  0.524463     0.034964   

                  epa   largest    modern    nation  ...     poor   process  \
finance      0.000000  0.000000  0.043703  0.000000  ...  0.00000  0.000000   
education    0.000000  0.000000  0.000000  0.000000  ...  0.58442  0.064936   
environment  0.454534  0.034964  0.000000  0.034964  ...  0.00000  0.000000   

              protect  revolution      rich      rule       set      step  \
finance      0.000000    0.000000  0.000000  0.917771  0.043703  0.033238   
education    0.000000    0.064936  0.129871  0.000000  0.000000  0.049385   
environment  0.524463    0.000000  0.000000  0.00

In [45]:
print("Normalized TF-IDF (Term Frequency-Inverse Document Frequency) for each word in all documents:")
for index, row in normalized_tf_idf_df.iterrows():
    values_line = '     '.join([f"{value:.8f}" for value in row.values])
    print( "[  "+values_line+" ]")


Normalized TF-IDF (Term Frequency-Inverse Document Frequency) for each word in all documents:
[  0.00000000     0.36561283     0.00000000     0.13111007     0.00000000     0.00000000     0.00000000     0.00000000     0.04370336     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.91777052     0.04370336     0.03323753     0.00000000     0.04370336 ]
[  0.12987101     0.04938512     0.77922604     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.06493550     0.58441953     0.06493550     0.00000000     0.06493550     0.12987101     0.00000000     0.00000000     0.04938512     0.00000000     0.00000000 ]
[  0.00000000     0.00000000     0.00000000     0.00000000     0.52446270     0.03496418     0.45453434     0.03496418     0.00000000     0.03496418     0.00000000     0.00000000     0.00000000     0.52446270     0.00000000     0.00000000     0.00000000     0.00000000     0

# **TF IDF built in**

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF calculation using scikit-learn
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_documents)

print("TF-IDF Matrix (scikit-learn):\n", tfidf_matrix.toarray())

TF-IDF Matrix (scikit-learn):
 [[0.         0.36561283 0.         0.13111007 0.         0.
  0.         0.         0.04370336 0.         0.         0.
  0.         0.         0.         0.         0.91777052 0.04370336
  0.03323753 0.         0.04370336]
 [0.12987101 0.04938512 0.77922604 0.         0.         0.
  0.         0.         0.         0.         0.0649355  0.58441953
  0.0649355  0.         0.0649355  0.12987101 0.         0.
  0.04938512 0.         0.        ]
 [0.         0.         0.         0.         0.5244627  0.03496418
  0.45453434 0.03496418 0.         0.03496418 0.         0.
  0.         0.5244627  0.         0.         0.         0.
  0.         0.48949852 0.        ]]


**Different way From scratch using normalize from sklearn**

In [49]:
from sklearn.preprocessing import normalize
import numpy as np

total_docs = len(preprocessed_documents)
word_idf = {}
for word in count_vectorizer.vocabulary_:
    word_docs_count = sum(1 for doc in preprocessed_documents if word in doc)
    word_idf[word] = total_docs / (1 + word_docs_count)

# Convert term_freq_matrix to float64
term_freq_matrix = term_freq_matrix.astype(np.float64)

# Multiply TF * IDF
tfidf_matrix_scratch = term_freq_matrix.copy()
for word, idx in count_vectorizer.vocabulary_.items():
    tfidf_matrix_scratch[:, idx] *= word_idf[word]

# Normalize TF-IDF
tfidf_matrix_scratch = normalize(tfidf_matrix_scratch)

print("\nTF-IDF Matrix (from scratch):\n", tfidf_matrix_scratch.toarray())




TF-IDF Matrix (from scratch):
 [[0.         0.32561342 0.         0.13320549 0.         0.
  0.         0.         0.04440183 0.         0.         0.
  0.         0.         0.         0.         0.93243842 0.04440183
  0.02960122 0.         0.04440183]
 [0.12994442 0.04331481 0.77966655 0.         0.         0.
  0.         0.         0.         0.         0.06497221 0.58474991
  0.06497221 0.         0.06497221 0.12994442 0.         0.
  0.04331481 0.         0.        ]
 [0.         0.         0.         0.         0.5244627  0.03496418
  0.45453434 0.03496418 0.         0.03496418 0.         0.
  0.         0.5244627  0.         0.         0.         0.
  0.         0.48949852 0.        ]]
